In [1]:
!pip install pytorch_lightning
!pip install transformers
!pip install --quiet sense2vec==1.0.3
!wget https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz
!tar -xvf  s2v_reddit_2015_md.tar.gz
!ls s2v_old
!pip install --quiet sentence_transformers==2.2.0
!pip install -U sentence-transformers
!pip install scipy
!pip install sense2vec
!pip install tika
!pip install aspose-words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 798 kB 35.5 MB/s 
     |████████████████████████████████| 529 kB 90.2 MB/s 
     |████████████████████████████████| 87 kB 8.1 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115940 sha256=6d2c8064318e84ee3c2bafc0cbcb16c7c14b97974f3b2123883dea0a73f8db31
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 27.5 MB/s 
     |████████████████████████████████| 7.6 MB 53.1 MB/s 
     |████████████████████████████████| 182 kB 88.1 MB/s 
     |████████████████████████████████| 4.8 MB 71.2 MB/s 
     |████████████████████████████████| 208 kB 85.9 MB/s 
     |████████████████████████████████| 1.0 MB 68.6 MB/s 


In [2]:
import aspose.words as aw
from typing import List, Dict
import tqdm.notebook as tq
import string
import itertools
from tqdm.notebook import tqdm
import json
import pandas as pd
import numpy as np
from transformers import pipeline
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sense2vec import Sense2Vec
from collections import OrderedDict
from platform import dist
import re
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from typing import List
from nltk.tokenize import sent_tokenize
from multiprocessing import Pool
from multiprocessing import cpu_count
from concurrent.futures import ProcessPoolExecutor
import toolz
from datetime import datetime
import requests
import random
import nltk
import textwrap
import time
from datetime import date
from tika import parser
import en_core_web_sm
import scipy
# from sense2vec import Sense2Vec
# s2v = Sense2Vec().from_disk('s2v_old')
# import en_core_web_sm
# import random
# import spacy.cli
# spacy.cli.download("en_core_web_sm")
# spacy_nlp = en_core_web_sm.load()
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer
    )
from sentence_transformers import SentenceTransformer
model_mcq= SentenceTransformer('all-MiniLM-L12-v2')
model_BERT = SentenceTransformer('bert-base-nli-mean-tokens')

/usr/local/lib/python3.7/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (3.4.1) requires spaCy v3.4 and is incompatible with the current spaCy version (2.3.8). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/573 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [3]:
start = time.time()

In [4]:
question_answerer = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/261M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [5]:
MODEL_NAME = 't5-small'
SOURCE_MAX_TOKEN_LEN = 64
TARGET_MAX_TOKEN_LEN = 24
LEARNING_RATE = 0.0001
SOURCE_MAX_TOKEN_LEN = 512
TARGET_MAX_TOKEN_LEN = 64
SEP_TOKEN = '<sep>'
TOKENIZER_LEN = 32101 #after adding the new <sep> token
SOURCE_MAX_TOKEN_LEN = 300
TARGET_MAX_TOKEN_LEN = 80
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
class QGModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
        self.model.resize_token_embeddings(TOKENIZER_LEN) #resizing after adding new tokens to the tokenizer

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('train_loss', loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('val_loss', loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('test_loss', loss, prog_bar=True, logger=True)
        return loss
  
    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=LEARNING_RATE)


class AnswerGenerator():
    def __init__(self):
        self.tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

        checkpoint_path = 'app/ml_models/answer_generation/models/squad-answer-generation.ckpt'
        self.ag_model = QGModel.load_from_checkpoint(checkpoint_path)
        self.ag_model.freeze()
        self.ag_model.eval()

    def generate(self, context, generate_count):
        model_output = self._model_predict(context, generate_count)

        answers = model_output.replace('<pad>', '').split('</s>')[:-1]

        return answers

    def _model_predict(self, context: str, generate_count: int) -> str:
        source_encoding = self.tokenizer(
            context,
            max_length=SOURCE_MAX_TOKEN_LEN,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        generated_ids = self.ag_model.model.generate(
            input_ids=source_encoding['input_ids'],
            attention_mask=source_encoding['attention_mask'],
            num_beams=generate_count,
            num_return_sequences=generate_count,
            max_length=TARGET_MAX_TOKEN_LEN,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            use_cache=True
        )

        preds = {
            self.tokenizer.decode(generated_id, skip_special_tokens=False, clean_up_tokenization_spaces=True)
            for generated_id in generated_ids
        }

        return ''.join(preds)



In [7]:
class DistractorGenerator():
    def __init__(self):
        self.tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
        # print('tokenizer len before: ', len(self.tokenizer))
        self.tokenizer.add_tokens(SEP_TOKEN)
        # print('tokenizer len after: ', len(self.tokenizer))
        self.tokenizer_len = len(self.tokenizer)

        checkpoint_path = '/content/drive/MyDrive/Leaf-Question-Generation-main/app/ml_models/distractor_generation/models/race-distractors.ckpt'
        self.dg_model = QGModel.load_from_checkpoint(checkpoint_path)
        self.dg_model.freeze()
        self.dg_model.eval()

    def generate(self, generate_count, correct, question, context):
        
        generate_triples_count = int(generate_count / 3) + 1 #since this model generates 3 distractors per generation
        
        model_output = self._model_predict(generate_triples_count, correct, question, context)

        cleaned_result = model_output.replace('<pad>', '').replace('</s>', '<sep>')
        cleaned_result = self._replace_all_extra_id(cleaned_result)
        distractors = cleaned_result.split('<sep>')[:-1]
        distractors = [x.translate(str.maketrans('', '', string.punctuation)) for x in distractors]
        distractors = list(map(lambda x: x.strip(), distractors))

        return distractors

    def _model_predict(self, generate_count: int, correct: str, question: str, context: str) -> str:
        source_encoding = self.tokenizer(
            '{} {} {} {} {}'.format(correct, SEP_TOKEN, question, SEP_TOKEN, context),
            max_length= SOURCE_MAX_TOKEN_LEN,
            padding='max_length',
            truncation= True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
            )

        generated_ids = self.dg_model.model.generate(
            input_ids=source_encoding['input_ids'],
            attention_mask=source_encoding['attention_mask'],
            num_beams=generate_count,
            num_return_sequences=generate_count,
            max_length=TARGET_MAX_TOKEN_LEN,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            use_cache=True
        )

        preds = {
            self.tokenizer.decode(generated_id, skip_special_tokens=False, clean_up_tokenization_spaces=True)
            for generated_id in generated_ids
        }

        return ''.join(preds)

    def _correct_index_of(self, text:str, substring: str, start_index: int = 0):
        try:
            index = text.index(substring, start_index)
        except ValueError:
            index = -1

        return index

    def _replace_all_extra_id(self, text: str):
        new_text = text
        start_index_of_extra_id = 0

        while (self._correct_index_of(new_text, '<extra_id_') >= 0):
            start_index_of_extra_id = self._correct_index_of(new_text, '<extra_id_', start_index_of_extra_id)
            end_index_of_extra_id = self._correct_index_of(new_text, '>', start_index_of_extra_id)

            new_text = new_text[:start_index_of_extra_id] + '<sep>' + new_text[end_index_of_extra_id + 1:]

        return new_text

    

In [8]:
class QuestionGenerator():
    def __init__(self):
        self.tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
        # print('tokenizer len before: ', len(self.tokenizer))
        self.tokenizer.add_tokens(SEP_TOKEN)
        # print('tokenizer len after: ', len(self.tokenizer))
        self.tokenizer_len = len(self.tokenizer)

        checkpoint_path = '/content/drive/MyDrive/Leaf-Question-Generation-main/app/ml_models/question_generation/models/multitask-qg-ag.ckpt'
        self.qg_model = QGModel.load_from_checkpoint(checkpoint_path)
        self.qg_model.freeze()
        self.qg_model.eval()

    def generate(self, answer: str, context: str) -> str:
        model_output = self._model_predict(answer, context)

        generated_answer, generated_question = model_output.split('<sep>')

        return generated_question

    def generate_qna(self, context):
        answer_mask = '[MASK]'
        model_output = self._model_predict(answer_mask, context)

        qna_pair = model_output.split('<sep>')

        if len(qna_pair) < 2:
            generated_answer = ''
            generated_question = qna_pair[0]
        else:
            generated_answer = qna_pair[0]
            generated_question = qna_pair[1]
  
        return generated_answer, generated_question, context

    def _model_predict(self, answer: str, context: str) -> str:
        source_encoding = self.tokenizer(
            '{} {} {}'.format(answer, SEP_TOKEN, context),
            max_length=SOURCE_MAX_TOKEN_LEN,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        generated_ids = self.qg_model.model.generate(
            input_ids=source_encoding['input_ids'],
            attention_mask=source_encoding['attention_mask'],
            num_beams=16,
            max_length=TARGET_MAX_TOKEN_LEN,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            use_cache=True
        )

        preds = {
            self.tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for generated_id in generated_ids
        }

        return ''.join(preds)


In [9]:
class Sense2VecDistractorGeneration():
    def __init__(self):
        self.s2v = Sense2Vec().from_disk('/content/drive/MyDrive/Leaf-Question-Generation-main/app/ml_models/sense2vec_distractor_generation/data/s2v_old')

    def generate(self, answer, desired_count):
        distractors = []
        answer = answer.lower()
        answer = answer.replace(" ", "_")

        sense = self.s2v.get_best_sense(answer)

        if not sense:
            return []

        most_similar = self.s2v.most_similar(sense, n=desired_count)

        for phrase in most_similar:
            normalized_phrase = phrase[0].split("|")[0].replace("_", " ").lower()

            if normalized_phrase.lower() != answer: #TODO: compare the stem of the words (e.g. wrote, writing)
                distractors.append(normalized_phrase.capitalize())

        return list(OrderedDict.fromkeys(distractors))


In [10]:
class Question:
    def __init__(self, answerText:str, questionText: str = '', distractors: List[str] = [], contextText:List[str] = ''):
        self.answerText = answerText
        self.questionText = questionText
        self.distractors = distractors
        self.contextText = contextText

In [11]:
def remove_duplicates(items: List[str]) -> List[str]:
    unique_items = []
    normalized_unique_items = []

    for item in items:


        normalized_item = _normalize_item(item)

        if normalized_item not in normalized_unique_items:
            unique_items.append(item)
            normalized_unique_items.append(normalized_item)

    return unique_items

def duplicate_question(questions_array):
    temp_arr = []
    questions_are = []
    for question in questions_array:
        if question["question"] not in temp_arr:
            temp_arr.append(question["question"])
            questions_are.append(question)
    return questions_are

def remove_distractors_duplicate_with_correct_answer(correct: str, distractors: List[str]) -> List[str]:
    for distractor in distractors:
        if _normalize_item(correct) == _normalize_item(distractor):
            distractors.remove(distractor)
    
    return distractors

'''
Code from: https://github.com/allenai/bi-att-flow/blob/master/squad/evaluate-v1.1.py
'''
def _normalize_item(item) -> str:
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(item))))

def _calculate_nltk_bleu(references: List[str], hypothesis: str, bleu_n: int = 1):
    if hypothesis == '': 
        return 0, 0, 0, 0 

    # Word tokenize
    refs_tokenized = list(map(lambda x: word_tokenize(x), references))
    hyp_tokenized = word_tokenize(hypothesis)

    # Smoothing function to avoid the cases where it resuts 1.0 in the cases when // Corpus/Sentence contains 0 counts of 2-gram overlaps. BLEU scores might be undesirable; use SmoothingFunction() //
    chencherry = SmoothingFunction()
    bleu = 0

    if bleu_n == 1:
        bleu = sentence_bleu(refs_tokenized, hyp_tokenized, weights=(1, 0, 0, 0), smoothing_function=chencherry.method2)
    elif bleu_n == 2:
        bleu = sentence_bleu(refs_tokenized, hyp_tokenized, weights=(0.5, 0.5, 0, 0), smoothing_function=chencherry.method2)
    elif bleu_n == 3: 
        bleu = sentence_bleu(refs_tokenized, hyp_tokenized, weights=(0.33, 0.33, 0.33, 0), smoothing_function=chencherry.method2)
    elif bleu_n == 4:
        bleu = sentence_bleu(refs_tokenized, hyp_tokenized, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=chencherry.method2)

    return bleu

In [12]:
def clean_text(text):
    cleaned_text = _remove_brackets(text)
    cleaned_text = _remove_square_brackets(cleaned_text)
    cleaned_text = _remove_multiple_spaces(cleaned_text)
    cleaned_text = _replace_weird_hyphen(cleaned_text)
    print(cleaned_text)
    return cleaned_text
    

def _remove_brackets(text: str) -> str:

    return re.sub(r'\((.*?)\)', lambda L: '', text)


def _remove_square_brackets(text: str) -> str:

    return re.sub(r'\[(.*?)\]', lambda L: '', text)


def _remove_multiple_spaces(text: str) -> str:

    return re.sub(' +', ' ', text)


def _replace_weird_hyphen(text: str) -> str:

    return text.replace('–', '-')

In [ ]:
def ner_distractors(Text,answer): 

    def get_ner_text(text):                                              # get NER label& words from the entire text
        sentences = sent_tokenize(str(text))
        sentence_to_words=[]
        for i in sentences[0:]:
            sentence_to_words.extend(i.split())  
        tokens = [ w for w in sentence_to_words if w[0]!='[' and w[-1]!= ']' ]
        import string
        remove = string.punctuation
        import re 
        remove = remove.replace(".", "").replace("-", "").replace(",", "").replace("%", "").replace("/", "")
        pattern = r"[{}]".format(re.escape(remove))
        table = str.maketrans('', '', pattern)
        stripped = [w.translate(table) for w in tokens]
        words_to_sentense=' '.join(stripped)
        ner_word = []
        text1= spacy_nlp(str(words_to_sentense))
        for word in text1.ents:
            ner_word.append({'word':word.text,'label':word.label_})
        return ner_word
  
    def get_answer_and_distractor_embeddings(answer,candidate_distractors):
        answer_embedding = model_mcq.encode([answer])
        distractor_embeddings = model_mcq.encode(candidate_distractors)
        return answer_embedding,distractor_embeddings

    ner_word=get_ner_text(Text)

    Answers=[]

    lab = spacy_nlp(answer)
    for word in lab.ents:
        Answers.append({'word':word.text,'label':word.label_})

    final_list=[]
    for i in Answers:
        label=i.get('label')
        answer=i.get('word')
        op=[]
        for j in ner_word:
            label2=j.get('label')
            if label==label2:
                output=j.get('word')
                op.append(output)
        final_list.append({'answer':answer,'options':op})
        
    Input_answer=[]
    Input_options=[]
    for i in final_list:
        x=i.get('answer')
        Input_answer.append(x)
        y=i.get('options')
        Input_options.append(y)

    Final_options=[]
    for i in Input_options:
        m=set(i)
        n = list(m)
        Final_options.append(n)

    MCQ = []
    dissimilar_sentences = []
    for i in Input_answer:
       
        originalword = i
        distractors=Final_options[Input_answer.index(i)]
        answer_embedd, distractor_embedds = get_answer_and_distractor_embeddings(originalword,distractors)
        distances = scipy.spatial.distance.cdist(answer_embedd,distractor_embedds , "cosine")[0]
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])

        for distance_indx, distance in results:
            dissimilar_sentences.append(distractors[distance_indx])

    return dissimilar_sentences

In [ ]:
def get_answer_and_distractor_embeddings(answer,candidate_distractors):
    answer_embedding = model_mcq.encode([answer])
    distractor_embeddings = model_mcq.encode(candidate_distractors)
    return answer_embedding,distractor_embeddings

In [13]:
class MCQGenerator():
    def __init__(self, is_verbose=False):
        start_time = time.perf_counter()
        print('Loading ML Models...')

        self.question_generator = QuestionGenerator()
        print('Loaded QuestionGenerator in', round(time.perf_counter() - start_time, 2), 'seconds.') if is_verbose else ''

        self.distractor_generator = DistractorGenerator()
        print('Loaded DistractorGenerator in', round(time.perf_counter() - start_time, 2), 'seconds.') if is_verbose else ''

        self.sense2vec_distractor_generator = Sense2VecDistractorGeneration()
        print('Loaded Sense2VecDistractorGenerator in', round(time.perf_counter() - start_time, 2), 'seconds.') if is_verbose else ''

    # Main function
    def generate_mcq_questions(self, context, desired_count) -> List[Question]:
        cleaned_text =  clean_text(context)
        questions = self._generate_question_answer_pairs(cleaned_text, desired_count)
        questions = self._generate_distractors(cleaned_text, questions)

        return questions

    def _generate_answers(self, context: str, desired_count: int) -> List[Question]:
        # answers = self.answer_generator.generate(context, desired_count)
        answers = self._generate_multiple_answers_according_to_desired_count(context, desired_count)

        print(answers)
        unique_answers = remove_duplicates(answers)

        questions = []
        for answer in unique_answers:
            questions.append(Question(answer))

        return questions

    def _generate_questions(self, context: str, questions: List[Question]) -> List[Question]:        
        for question in questions:
            question.questionText = self.question_generator.generate(question.answerText, context)

        return questions

    def _generate_question_answer_pairs(self, context, desired_count):
        context_splits = self._split_context_according_to_desired_count(context, desired_count)
  
        questions = []

        for split in context_splits:
            answer, question, context = self.question_generator.generate_qna(split)
            questions.append({"question" :question,"answer":answer.capitalize(),"context": context})

        # questions = list(toolz.unique(questions, key=lambda x: x.answerText))
        
        return questions

    def _generate_distractors(self, context, questions):
        for question in questions:
            t5_distractors =  self.distractor_generator.generate(5, question['answer'], question["question"], context)
            s2v_distractors = self.sense2vec_distractor_generator.generate(question['answer'], 10)
            distractors = t5_distractors + s2v_distractors
            distractors = remove_duplicates(distractors)
            distractors = remove_distractors_duplicate_with_correct_answer(question['answer'], distractors)
            question["distractors"] =distractors
        return questions

    # Helper functions 
    def _generate_answer_for_each_sentence(self, context: str) -> List[str]:
        sents = sent_tokenize(context)

        answers = []
        for sent in sents:
            answers.append(self.answer_generator.generate(sent, 1)[0])

        return answers

    #TODO: refactor to create better splits closer to the desired amount
    def _split_context_according_to_desired_count(self, context: str, desired_count: int) -> List[str]:
        sents = sent_tokenize(context)
        sent_ratio = len(sents) / desired_count

        context_splits = []

        if sent_ratio < 1:
            return sents
        else:
            take_sents_count = int(sent_ratio + 1)

            start_sent_index = 0

            while start_sent_index < len(sents):
                context_split = ' '.join(sents[start_sent_index: start_sent_index + take_sents_count])
                context_splits.append(context_split)
                start_sent_index += take_sents_count - 1

        return context_splits
    


In [ ]:
def add_noa_option(data,user_required_data):
    length_total_data = len(data)
    if user_required_data != "default":
        user_required_data = int(user_required_data)
        length_user_required_total_data = round((user_required_data*length_total_data)/100)
        user_data = random.sample(data, k=length_user_required_total_data)
        length_first_wrong_data = round((50*len(user_data))/100)
        first_wrong_data = random.sample(user_data, k=length_first_wrong_data)

        replaced_first_wrong_data = replace_wrong_data(first_wrong_data)

        second_right_data = []
        for i in user_data:
            if i not in first_wrong_data:
                second_right_data.append(i)
        replaced_second_right_data = replace_right_data(second_right_data)

        other_user_data = []
        for i in data:
            if i not in user_data:
                other_user_data.append(i)

        final_data = get_final_data(replaced_second_right_data, replaced_first_wrong_data, other_user_data)
        return final_data
    else:
        if length_total_data == 2:
            length_small_data  = (50*length_total_data)//100
            small_data_replace = random.sample(data, k=length_small_data)       
            get_replaced_small_wrong_data = replace_wrong_data(small_data_replace)

            small_other_data = []
            for i in data:
                if i not in small_data_replace:
                    small_other_data.append(i)

            final_data = get_final_data(list(), get_replaced_small_wrong_data, small_other_data)
            return final_data

        else:
            length_first_half_data = (40*length_total_data)//100
            data_replace = random.sample(data, k=length_first_half_data)

            if length_first_half_data == 1: 
                wrong_data = random.sample(data_replace, k=length_first_half_data)

                other_second_half = []
                for i in data:
                    if i not in data_replace:
                        other_second_half.append(i)

                get_replaced_second_wrong_data = replace_wrong_data(wrong_data)

                final_data = get_final_data(list(), get_replaced_second_wrong_data, other_second_half)
                return final_data

            else: 
                replace_half  = (50*length_first_half_data)//100 
                wrong_data = random.sample(data_replace, k=replace_half) 
                get_replaced_wrong_data = replace_wrong_data(wrong_data)

                right_data = []
                for i in data_replace:
                    if i not in wrong_data:
                        right_data.append(i)

                get_replaced_right_data = replace_right_data(right_data)

                other_data = []
                for i in data:
                    if i not in data_replace:
                        other_data.append(i)

                final_data = get_final_data(get_replaced_right_data, get_replaced_wrong_data, other_data)
                return final_data
        
def replace_right_data(data):
    for j in data:
        if "None of the above" not in j["options"]:
            for option_index, option in enumerate(j["options"]):
                if option.capitalize() == j["answer"] or option == j["answer"]:
                    j["options"][option_index] = "None of the above"
                    j["correct_answer"] = j["answer"]
                    j["answer"] =  j["options"][option_index]
    return data

def replace_wrong_data(data):
    for j in data:
        if "None of the above" not in j["options"]:
            for option_index, option in enumerate(j["options"]):
                option = random.choice(j["options"])
                if option.capitalize() != j["answer"] or option != j["answer"]:
                    j["options"][option_index] = "None of the above"
                    random.shuffle(j["options"])
                    break
        else:
            print("yes data present")
    return data

def get_final_data(data_right, data_wrong, data_other):
    final_data1 = data_right + data_wrong + data_other
    final_data1 = sorted(final_data1, key=lambda i: float(i['rank']),reverse=True)
    
    final_data2=[]
    for i in final_data1:
      rank=float(i["rank"])
      if rank <= float(10):
        del i
      else:
        final_data2.append(i)
    return final_data2
    

In [14]:
def Sorting(new_questions):
  final_data = sorted(new_questions, key=lambda i: float(i['rank']),reverse=True)
  return final_data

def flatten(list_of_lists):
    return list(itertools.chain.from_iterable(list_of_lists))

In [15]:
def questions_with_mcq_rank(m):

    descriptive_MCQ=[] 
    multiple_choice=[]
    for index,i in enumerate(m):
        distractor=i['distractors']
        if len(distractor)==0 or len(distractor)==1 :
            descriptive_MCQ.append(i)
        else:
            multiple_choice.append(i)
    for i in multiple_choice:
        if len(i['distractors']) >2:
            options = i['distractors'][0:3] + [i["answer"]]
            random.shuffle(options)  
            i.update({'options':options})          
        else:
            options = i['distractors'][0:2] + [i["answer"]] 
            random.shuffle(options)
            options=options+["None of the above"]
            i.update({'options':options}) 
    for i in multiple_choice:
        del i["distractors"]
        final_score_list = question_answerer(question=i['question'], context=i['context'])
        final_score = final_score_list.get("score")
        final_score ="%.2f" % round(final_score*100, 2)
        i.update({'rank':final_score})

    return multiple_choice,descriptive_MCQ

In [ ]:
request=requests.get("https://generate-questions.devbyopeneyes.com/api/getFileData/637617217ea815b43001f272")
resp= request.json()
file_path=(resp["data"]["file_path"])
type_of_question=(resp["data"]["type_of_question"])
_id=(resp["data"]["_id"])
number_of_question=(resp["data"]["number_of_question"])
file_type=(resp["data"]["file_type"])

In [ ]:
if file_type == "pdf":
  FileName=file_path
  doc = aw.Document(FileName)
  doc.save("File.docx")
  Parse=parser.from_file("File.docx")
  data=[]
  if not Parse['content'] is None:
    for i in (Parse['content'].strip().split('\n')): 
      if len(i.split())<5:
        pass
      else: 
        data.append(i)
    Text=data[1:-1]
  else:
    Text = None
else:
  FileName=file_path
  Parse=parser.from_file(FileName)
  data=[] 
  if not Parse['content'] is None:
    for i in (Parse['content'].strip().split('\n')): 
      if len(i.split())<5:
        pass
      else: 
        data.append(i)
    Text=data
  else:
    Text = None

2022-11-17 11:15:23,324 [MainThread  ] [INFO ]  Retrieving https://drive.google.com/u/0/uc?id=18Khc12v5dmJmun7POQuUAJWBpk9nKcvy&export=download to /tmp/u-0-uc.
INFO:tika.tika:Retrieving https://drive.google.com/u/0/uc?id=18Khc12v5dmJmun7POQuUAJWBpk9nKcvy&export=download to /tmp/u-0-uc.


In [ ]:
if Text == None or len(Text) == 0:
  print("please enter valid file")
else:
  print("valid file")

valid file


In [16]:
MCQ_Generator = MCQGenerator(True)

Loading ML Models...


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

Loaded QuestionGenerator in 16.13 seconds.
Loaded DistractorGenerator in 26.57 seconds.
Loaded Sense2VecDistractorGenerator in 46.84 seconds.


In [17]:
def Pipeline_MCQ(text):
    a=MCQ_Generator.generate_mcq_questions(text,100)        ## for generating questions
    b=questions_with_mcq_rank(a)                      ## generating distractors 
    return b[0]                                             # b[1] to get descriptive questions

In [18]:
def Pipeline_WH(text):
    wh_questions = []
    questions_are = MCQ_Generator.generate_mcq_questions(text,20) 
    all_question = generate_questions_with_des(questions_are)
    for i in all_question:
        del i["options"]
        wh_questions.append(i)
    return wh_questions

In [19]:
 
def listToString(s):
    str1 = ""
    for ele in s:
        str1 += ele
    return str1

In [20]:
text_new='''The parable begins with a man who had two sons, and the younger of them asks his father to give him his share of the estate.
The implication is the son could not wait for his father's death for his inheritance, he wanted it immediately.
The father agrees and divides his estate between both sons.
I will arise and go to my father, and will say unto him, Father, I have sinned against heaven, and before thee, and am no more worthy to be called thy son: make me as one of thy hired servants.
This implies the father was hopefully watching for the son's return.
The son starts his rehearsed speech, admitting his sins, and declaring himself unworthy of being his father's son, but in most versions of Luke, the son does not even finish, before his father accepts him back wholeheartedly without hesitation as the father calls for his servants to dress the son in the finest robe available, get a ring for his finger, and sandals for his feet, and to slaughter the "fatted calf" for a celebratory meal.
The older son, who was at work in the fields, hears the sound of celebration, and is told by a fellow servant about the return of his younger brother.
He also has a speech for his father.
And he answering said to his father, Lo, these many years do I serve thee, neither transgressed I at any time thy commandment: and yet thou never gavest me a kid, that I might make merry with my friends: but as soon as this thy son was come, which hath devoured thy living with harlots, thou hast killed for him the fatted calf.
The parable concludes with the father explaining that while the older son has always been present, and everything the father owns also belongs to the older son, because the younger son had returned, in a sense, from the dead, celebration was necessary.'''

In [22]:
type_of_question = "MCQ" 

In [23]:
get_paragraph = []
  
if type_of_question == "MCQ" : 
    m= Pipeline_MCQ(text_new) # Flattening nested list
    final_generated_questions=Sorting(m)
            
elif type_of_question == "WH":
    result =[]
    with ProcessPoolExecutor(max_workers=2) as exe:
      exe.submit(Pipeline_WH,2)
      result = exe.map(Pipeline_WH,Text)

    outputs=list(result)

    x=1
    Final_list=[]  
    for i in outputs:
      Final_new=[]
      for j in i:
        j['Paragraph_no']=x
        Final_new.append(j)
      x+=1
      Final_list.append(Final_new)

    m= flatten(Final_list) # Flattening nested list
    final_generated_questions=Sorting(m)
else:
        print("Invalid Question type, please enter valid type")

The parable begins with a man who had two sons, and the younger of them asks his father to give him his share of the estate.
The implication is the son could not wait for his father's death for his inheritance, he wanted it immediately.
The father agrees and divides his estate between both sons.
I will arise and go to my father, and will say unto him, Father, I have sinned against heaven, and before thee, and am no more worthy to be called thy son: make me as one of thy hired servants.
This implies the father was hopefully watching for the son's return.
The son starts his rehearsed speech, admitting his sins, and declaring himself unworthy of being his father's son, but in most versions of Luke, the son does not even finish, before his father accepts him back wholeheartedly without hesitation as the father calls for his servants to dress the son in the finest robe available, get a ring for his finger, and sandals for his feet, and to slaughter the "fatted calf" for a celebratory meal.


In [24]:
final_generated_questions

[{'question': " Who was hoping for the son's return?",
  'answer': 'The father',
  'context': "This implies the father was hopefully watching for the son's return.",
  'options': ['Luke', 'The father', 'The servant', 'None of the above'],
  'rank': '93.23'},
 {'question': ' Who agrees and divides his estate between two sons?',
  'answer': 'The father',
  'context': 'The father agrees and divides his estate between both sons.',
  'options': ['The younger',
   'The father',
   'The older son',
   'None of the above'],
  'rank': '91.86'},
 {'question': ' Who explained that while the older son has always been present, and everything the father owns also belongs to the older son?',
  'answer': 'Father',
  'context': 'The parable concludes with the father explaining that while the older son has always been present, and everything the father owns also belongs to the older son, because the younger son had returned, in a sense, from the dead, celebration was necessary.',
  'options': ['Stepfath

In [ ]:
total_len_question = len(final_generated_questions)

In [ ]:
end = time.time()

In [ ]:
dt1 = datetime.fromtimestamp(start)
dt2 = datetime.fromtimestamp(end)
total_time = dt2 - dt1

In [ ]:
number_of_question = int(number_of_question)
if number_of_question > len(final_generated_questions) or number_of_question == 0:
  print("please give value between 1 to {}".format(len(final_generated_questions)))
else:
    middle_index = number_of_question
    user_required_questions = final_generated_questions[:middle_index]
    other_questions = final_generated_questions[middle_index:]
   
    for question_index, question in enumerate(user_required_questions):
      question["question_id"]= question_index +1

    for question_index, question in enumerate(other_questions):
      question["question_id"]= question_index +1

    generated_date = date.today()
    url="https://generate-questions.devbyopeneyes.com/api/GenerateQuestions" 
    headers = {'Content-Type':'application/json','Accept':'application/json'}
    post_array ={
                "id" : _id,
                "questions" : user_required_questions,
                "other_questions" : other_questions,
                "upload_process_time": str(total_time),
                "generate_date": str(generated_date),
                "no_of_question_generate":str(total_len_question)
            }
    status = requests.post(url,headers=headers,data=json.dumps(post_array))

please give value between 1 to 9
